<a href="https://colab.research.google.com/github/pravin691983/insaid-capstone_project_1/blob/development/SourceCode/insaid_capstone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSAID - Capstone Project on Data Analytics**


# Problem Statement

**InsaidTelecom**, one of the leading telecom players, understands that customizing offering
is very important for its business to stay competitive.
Currently, InsaidTelecom is seeking to leverage behavioral data from more than 60% of the 50 million mobile devices active daily in India
to help its clients better understand and interact with their audiences.



# Objective

In this **consulting assignment**, Insaidians are expected to build a dashboard to understand
user's demographic characteristics based on their mobile usage, geolocation, and mobile device properties.
Doing so will help millions of developers and brand advertisers around the world pursue
data-driven marketing efforts which are relevant to their users and catered to their preferences.

To help the customer the consultants are expected to have depth of clarity in the underlying data.
How much effort has been put into cleansing and purifying the data will decide how closely have you looked at the data.
How detailed is the observation stated in the submission report and finally how well a group presents their consulting journey.

Please remember this is an analytics consulting hence, your efforts in terms of
finding user behavior is going to directly impact the company's offerings.
Do help the company understand what is the
right way forward and suggest actionable insights from marketing and product terms.

# Data Loading and Description

In this assignment, we are going to study the demographics of a user (gender and age) based on their app download and usage behaviors.
The Data is collected from mobile apps that use InsaidTelecom services. Full recognition and consent from individual user of those apps have been obtained,
and appropriate anonymization have been performed to protect privacy. Due to confidentiality, we won't provide details on how the gender and age data was obtained. <Br>


The data schema can be represented in the following table: <BR>

**gender_age_train** - Devices and their respective user gender, age and age_group

**phone_brand_device_model** - device ids, brand, and models phone_brand: note that few brands are in Chinese

// TODO :  added Table of devices list with brands

**events_data** - when a user uses mobile on INSAID Telecom network, the event gets logged in this data.
Each event has an event id, location (lat/long), and the event corresponds to frequency of mobile usage.
timestamp: when the user is using the mobile.

Download the DataSets for events_data from : [events_data.csv](https://https://drive.google.com/uc?export=download&id=1OK4yi-f1v03vS6fBMjuOLKPhNOfjzsh3)

Download the DataSets for **gender_age_train** & **phone_brand_device_model** onto Python by connecting to the below
provided MySQL instance.



---



***Importing a library that is not in Colaboratory*** 
<Br>
To import a library that's not in Colaboratory by default, you can use !pip install or !apt-get install.

In [0]:
!pip install -q mysql-connector-python-rf

In [0]:
#Import Required packages for MySQL instance
import mysql.connector
from mysql.connector import Error

**Importing the Dataset**

In [0]:
import numpy as np                                                 # Implemennts milti-dimensional array and matrices
import pandas as pd                                                # For data manipulation and analysis
import pandas_profiling
import matplotlib.pyplot as plt                                    # Plotting library for Python programming language and it's numerical mathematics extension NumPy
import seaborn as sns                                              # Provides a high level interface for drawing attractive and informative statistical graphics

%matplotlib inline
sns.set()

from subprocess import check_output

# Data Profiling


*   In the upcoming sections we will first understand our dataset using various pandas functionalities.
*   Then with the help of pandas profiling we will find which columns of our dataset need preprocessing.
*   In preprocessing we will deal with erronous and missing values of columns.
*   Again we will do pandas profiling to see how preprocessing have transformed our dataset.





**3.1 Understanding the Dataset**

To gain insights from data we must look into each aspect of it very carefully. We will start with observing few rows and columns of data both from the starting and from the end.

Let us check the basic information of the dataset. The very basic information to know is the dimension of the dataset – rows and columns – that’s what we find out with the method shape.

In [0]:
print("Understanding the Dataset")

**3.2 Pre Profiling**

By pandas profiling, **an interactive HTML report** gets generated which contins all the information about the columns of the dataset, like the counts and type of each column. Detailed information about each column, **correlation between different columns** and a sample of dataset.<Br>


*   It gives us **visual interpretation** of each column in the data.
*   Spread of the data can be better understood by the distribution plot.
*   Grannular level analysis of each column.

In [0]:
profile = pandas_profiling.ProfileReport(matches_data)
profile.to_file(outputfile="data_before_preprocessing.html")

Here, we have done Pandas Profiling before preprocessing our dataset, so we have named the html file as **data_before_preprocessing.html**. <Br>
Take a look at the file and see what useful insight you can develop from it.
Now we will process our data to better understand it.

**3.3 Preprocessing**


// TODO: Preprocessing

**3.4 Post Pandas Profiling**

Now we have preprocessed the data, now the dataset doesnot contain missing values. <BR>
You can compare the two reports, i.e **data_after_preprocessing.html** and **data_before_preprocessing.html**.


In **data_after_preprocessing.html** report, observations:

In the Dataset info, 
* Total Missing(%) = 0.0%
* Number of variables = 13

# Questions

// TODO: Add Question List

# Conclusion

// TODO :